# ⚛️ H₂ VQE — Noisy Optimizer Comparison (Pure Package Client)

This notebook compares **classical optimizers** for **H₂ VQE** under noise using only:

```python
from vqe.core import run_vqe
```

For each optimizer and each noise level, we:
1) compute a noiseless reference energy (cached) per seed
2) run noisy VQE per seed
3) report mean/std of ΔE = E_noisy − E_ref across seeds

Noise modes:
- Depolarizing sweep (amp=0)
- Amplitude-damping sweep (dep=0)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from vqe.core import run_vqe

## Configuration

In [ ]:
molecule = "H2"
ansatz_name = "UCCSD"
mapping = "jordan_wigner"

# Noise grid in [0.00, 0.10]
noise_levels = np.arange(0.0, 0.11, 0.02)

# Multi-seed statistics
seeds = np.arange(0, 10)

# Optimizers to compare
optimizers = ["Adam", "GradientDescent", "Momentum", "Nesterov", "RMSProp", "Adagrad"]

# Optimization parameters
steps = 75
stepsize_map = {
    "Adam": 0.2,
    "GradientDescent": 0.05,
    "Momentum": 0.1,
    "Nesterov": 0.1,
    "RMSProp": 0.1,
    "Adagrad": 0.2,
}

print("Molecule:", molecule)
print("Ansatz:", ansatz_name)
print("Mapping:", mapping)
print("Steps:", steps)
print("Noise levels:", noise_levels)
print("Seeds:", seeds)
print("Optimizers:", optimizers)
print("Stepsizes:", stepsize_map)

## Helper: run one configuration across seeds (returns ΔE stats)

In [ ]:
def _deltaE_stats_for_config(
    *,
    optimizer_name: str,
    stepsize: float,
    noise_type: str,
    noise_p: float,
) -> tuple[float, float]:
    """
    Returns (mean, std) of ΔE across seeds where:
      ΔE = E_noisy - E_ref
    """
    dEs = []

    for sd in seeds:
        # Noiseless reference (cached)
        ref = run_vqe(
            molecule=molecule,
            ansatz_name=ansatz_name,
            optimizer_name=optimizer_name,
            steps=int(steps),
            stepsize=float(stepsize),
            noisy=False,
            mapping=mapping,
            seed=int(sd),
            plot=False,
            force=False,
        )
        E_ref = float(ref.get("energy", ref.get("final_energy", np.nan)))
        if not np.isfinite(E_ref):
            raise RuntimeError(
                f"Reference run_vqe returned no finite energy for optimizer='{optimizer_name}', seed={sd}"
            )

        # Noisy run
        dep = float(noise_p) if noise_type == "depolarizing" else 0.0
        amp = float(noise_p) if noise_type == "amplitude" else 0.0

        out = run_vqe(
            molecule=molecule,
            ansatz_name=ansatz_name,
            optimizer_name=optimizer_name,
            steps=int(steps),
            stepsize=float(stepsize),
            noisy=True,
            depolarizing_prob=dep,
            amplitude_damping_prob=amp,
            mapping=mapping,
            seed=int(sd),
            plot=False,
            force=False,
        )
        E_noisy = float(out.get("energy", out.get("final_energy", np.nan)))
        if not np.isfinite(E_noisy):
            raise RuntimeError(
                f"Noisy run_vqe returned no finite energy for optimizer='{optimizer_name}', seed={sd}, p={noise_p}"
            )

        dEs.append(E_noisy - E_ref)

    dEs = np.asarray(dEs, dtype=float)
    mean = float(dEs.mean())
    std = float(dEs.std(ddof=1)) if len(dEs) > 1 else 0.0
    return mean, std

## Part 1 — Depolarizing noise sweep (amp = 0)

We compute mean/std of ΔE across seeds for each optimizer and each depolarizing probability.

In [ ]:
dep_means = {opt: [] for opt in optimizers}
dep_stds = {opt: [] for opt in optimizers}

for opt in optimizers:
    ss = float(stepsize_map[opt])
    print(f"\n[Depolarizing] Optimizer = {opt} (stepsize={ss})")

    for p in noise_levels:
        m, s = _deltaE_stats_for_config(
            optimizer_name=opt,
            stepsize=ss,
            noise_type="depolarizing",
            noise_p=float(p),
        )
        dep_means[opt].append(m)
        dep_stds[opt].append(s)
        print(f"  p={p:0.2f} -> ΔE mean={m:+.6f} std={s:.6f}")

### Plot — ΔE vs depolarizing probability (one curve per optimizer)

In [ ]:
plt.figure(figsize=(10, 6))
for opt in optimizers:
    plt.errorbar(
        noise_levels,
        np.asarray(dep_means[opt], dtype=float),
        yerr=np.asarray(dep_stds[opt], dtype=float),
        capsize=3,
        label=opt,
    )

plt.axhline(0.0, linewidth=1)
plt.xlabel("Depolarizing probability $p_{dep}$")
plt.ylabel("ΔE = E_noisy − E_ref (Ha)")
plt.title(f"{molecule} VQE — Optimizer Comparison under Depolarizing Noise ({len(seeds)} seeds)")
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.show()

## Part 2 — Amplitude damping sweep (dep = 0)

In [ ]:
amp_means = {opt: [] for opt in optimizers}
amp_stds = {opt: [] for opt in optimizers}

for opt in optimizers:
    ss = float(stepsize_map[opt])
    print(f"\n[Amplitude] Optimizer = {opt} (stepsize={ss})")

    for p in noise_levels:
        m, s = _deltaE_stats_for_config(
            optimizer_name=opt,
            stepsize=ss,
            noise_type="amplitude",
            noise_p=float(p),
        )
        amp_means[opt].append(m)
        amp_stds[opt].append(s)
        print(f"  p={p:0.2f} -> ΔE mean={m:+.6f} std={s:.6f}")

### Plot — ΔE vs amplitude damping probability (one curve per optimizer)

In [ ]:
plt.figure(figsize=(10, 6))
for opt in optimizers:
    plt.errorbar(
        noise_levels,
        np.asarray(amp_means[opt], dtype=float),
        yerr=np.asarray(amp_stds[opt], dtype=float),
        capsize=3,
        label=opt,
    )

plt.axhline(0.0, linewidth=1)
plt.xlabel("Amplitude damping probability $p_{amp}$")
plt.ylabel("ΔE = E_noisy − E_ref (Ha)")
plt.title(f"{molecule} VQE — Optimizer Comparison under Amplitude Damping ({len(seeds)} seeds)")
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.show()